In [18]:
import tensorflow as tf
import sagemaker

In [19]:
tf.__version__

'2.1.3'

In [20]:
import keras

In [21]:
import boto3
sagemaker_session = sagemaker.Session()

training_input_path = "s3://utec-workshop-cats-n-dogs/data/"
training_input_path

's3://utec-workshop-cats-n-dogs/data/'

In [22]:
tf_version = tf.__version__
tf_version

role = sagemaker.get_execution_role()
role

'arn:aws:iam::471361871242:role/service-role/AmazonSageMaker-ExecutionRole-20200822T095437'

In [23]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='train.py', 
                          role=role,
                          instance_count=1, 
                          instance_type='ml.g4dn.xlarge',
                          framework_version='2.4', 
                          py_version='py37',
                          script_mode=True,
                          hyperparameters={
                              'epochs': 1,
                          }
                         )

#Training
tf_estimator.fit({'train': training_input_path})

2021-04-16 13:12:27 Starting - Starting the training job...
2021-04-16 13:12:50 Starting - Launching requested ML instancesProfilerReport-1618578746: InProgress
......
2021-04-16 13:13:56 Starting - Preparing the instances for training.........
2021-04-16 13:15:10 Downloading - Downloading input data..................
2021-04-16 13:18:18 Training - Training image download completed. Training in progress..2021-04-16 13:18:19.520194: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-04-16 13:18:19.529294: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-04-16 13:18:19.698339: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-16 13:18:19.830272: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializi

In [24]:
import time
tf_endpoint_name = 'dog-cat-classifier'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
tf_predictor = tf_estimator.deploy(initial_instance_count=1,instance_type='ml.m5.4xlarge',
                                   endpoint_name=tf_endpoint_name)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!

In [25]:
from tensorflow.python.keras.preprocessing.image import load_img
import numpy as np

In [26]:
img = load_img("./test_images/9718.jpg", target_size=(256, 256))
img = np.array(img).reshape((1, 256, 256, 3))

In [27]:
img.shape

(1, 256, 256, 3)

In [28]:
predictions = tf_predictor.predict(img)['predictions'] #returns a list of lists

In [29]:
print("dog" if predictions[0][0]>0 else "Cat")

Cat
